In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!pip install folium
import folium # map rendering library

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.4 MB/s  eta 0:00:01
Libraries imported.


In [3]:
CLIENT_ID = 'O4V4WASPK5YOMVGX3TSHFOZ5NQAHDEJZ5F5IZYNGZMJE4U2K' # your Foursquare ID
CLIENT_SECRET = 'YMLF4WKHRZ4NUISGQOPY1D4XTUTIFDZ0LPQJKIFYS3UCE233' # your Foursquare Secret
VERSION = '20200830' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: O4V4WASPK5YOMVGX3TSHFOZ5NQAHDEJZ5F5IZYNGZMJE4U2K


In [4]:
# Getting the pizza places from foursquare for the big 5 cities

LIMIT = 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [6]:
#Defining the dataframe

df_venues={}
for city in cities:
    venues = pd.json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [7]:
#Checking the densities using map

maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in New York, NY =  287
Showing Top 100
Total number of pizza places in Chicago, IL =  220
Showing Top 100
Total number of pizza places in San Francisco, CA =  165
Showing Top 100
Total number of pizza places in Jersey City, NJ =  126
Showing Top 100
Total number of pizza places in Boston, MA =  185
Showing Top 100


In [8]:
maps[cities[0]]

In [9]:
maps[cities[1]]

In [10]:
maps[cities[2]]

In [11]:
maps[cities[3]]

In [12]:
maps[cities[4]]

In [13]:
#I will get the mean location of the pizza places which should be near to most of them if they are really dense or far if not. Next I will take the average of the distance of the venues to the mean coordinates.

for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.02355780980194641
Chicago, IL
Mean Distance from Mean coordinates
0.05736433542465731
San Francisco, CA
Mean Distance from Mean coordinates
0.028669570711800527
Jersey City, NJ
Mean Distance from Mean coordinates
0.01936578522692453
Boston, MA
Mean Distance from Mean coordinates
0.03590315981443015


In [14]:
maps[cities[0]]

In [15]:
maps[cities[1]]

In [16]:
maps[cities[2]]

In [17]:
maps[cities[3]]

In [18]:
maps[cities[4]]

It is clear that Jersey is the best oprion for a tourist to reserve his hotel near to the mean location and by that he will be surrounded by close 100 Pizza restaurants.